In [1]:
import tensorflow as tf

#创建一个常量迷，产生一个1x2的矩阵，这个op被做为一个节点加到默认图中
#构造器的返回值代表该 常量op的返回值
matrix1 = tf.constant([[3.,3.]])
#创建另一个常量op,产生一个2x1的矩阵
matrix2 = tf.constant([[2.],[2.]])
#创建一个矩阵乘法matmul op,把matrix1和matrix2作为输入
#返回值 product代表矩阵乘法的结果
product = tf.matmul(matrix1,matrix2)

In [4]:
#启动默认图
sess = tf.Session()

# 调用 sess 的 'run()' 方法来执行矩阵乘法 op, 传入 'product' 作为该方法的参数. 
# 上面提到, 'product' 代表了矩阵乘法 op 的输出, 传入它是向方法表明, 我们希望取回
# 矩阵乘法 op 的输出.
#
# 整个执行过程是自动化的, 会话负责传递 op 所需的全部输入. op 通常是并发执行的.
# 
# 函数调用 'run(product)' 触发了图中三个 op (两个常量 op 和一个矩阵乘法 op) 的执行.
#
# 返回值 'result' 是一个 numpy `ndarray` 对象.

result = sess.run(product)
print(result)

#任务完成 关闭会话
sess.close()

[[12.]]


In [5]:
with tf.Session() as sess:
    result = sess.run([product])
    print(result)

[array([[12.]], dtype=float32)]


### 交互式使用
文档中的 Python 示例使用一个会话 `Session` 来 启动图, 并调用 `Session.run()` 方法执行操作.

为了便于使用诸如 IPython 之类的 Python 交互环境, 可以使用 InteractiveSession 代替 Session 类, 使用` Tensor.eval()` 和` Operation.run()` 方法代替 `Session.run()`. 这样可以避免使用一个变量来持有会话.

In [8]:
# 进入一个交互式 TensorFlow 会话.
sess = tf.InteractiveSession()

x = tf.Variable([1.,2.])
a = tf.constant([3.,4.])


In [18]:
# 使用初始化器 initializer op 的 run() 方法初始化 'x' 
x.initializer.run()

# 增加一个减法 sub op, 从 'x' 减去 'a'. 运行减法 op, 输出结果 
sub = tf.add(x,a)
print(sub.eval())

[4. 6.]


### Tensor
TensorFlow 程序使用 tensor 数据结构来代表所有的数据, 计算图中, 操作间传递的数据都是 tensor. 你可以把 TensorFlow tensor 看作是一个 n 维的数组或列表. 一个 tensor 包含一个静态类型 rank, 和 一个 shape. 想了解 TensorFlow 是如何处理这些概念的, 参见 Rank, Shape, 和 Type.

### 变量
Variables for more details. 变量维护图执行过程中的状态信息. 下面的例子演示了如何使用变量实现一个简单的计数器. 参见 变量 章节了解更多细节.

In [13]:
# 创建一个变量, 初始化为标量 0.
state = tf.Variable(0,name="counter")

# 创建一个 op, 其作用是使 state 增加 1
one = tf.constant(1)
new_value = tf.add(state,one)
update = tf.assign(state,new_value)

# 启动图后, 变量必须先经过`初始化` (init) op 初始化,
# 首先必须增加一个`初始化` op 到图中.
init_op = tf.global_variables_initializer()

# 启动图, 运行 op
with tf.Session() as sess:
    #运行 'init' op
    sess.run(init_op)
    #打印 'state'的初始值
    print(sess.run(state))
    # 运行 op, 更新 'state', 并打印 'state'
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


### Fetch
为了取回操作的输出内容, 可以在使用 Session 对象的 run() 调用 执行图时, 传入一些 tensor, 这些 tensor 会帮助你取回结果. 在之前的例子里, 我们只取回了单个节点 state, 但是你也可以取回多个 tensor:

In [15]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2,input3)
mul = tf.multiply(input1,intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)

[21.0, 7.0]


### Feed
上述示例在计算图中引入了 tensor, 以常量或变量的形式存储. TensorFlow 还提供了 feed 机制, 该机制 可以临时替代图中的任意操作中的 tensor 可以对图中任何操作提交补丁, 直接插入一个 tensor.

feed 使用一个 tensor 值临时替换一个操作的输出结果. 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失. 最常见的用例是将某些特殊的操作指定为 "feed" 操作, 标记的方法是使用 tf.placeholder() 为这些操作创建占位符.

In [26]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1,input2)

with tf.Session() as sess:
    print(sess.run([output],feed_dict={input1:[7.],input2:[2.]}))

[array([14.], dtype=float32)]
